In [ ]:
from pyspark.sql import SparkSession
import re

spark_session = SparkSession.builder\
        .master("spark://192.168.2.250:7077") \
        .appName("A3_AN_Spark") \
        .config("spark.dynamicAllocation.enabled", True) \
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True) \
        .config("spark.shuffle.service.enabled", False) \
        .config("spark.dynamicAllocation.executorIdleTimeout", "30s") \
        .config("spark.executor.cores", 4) \
        .config("spark.driver.port", 9999) \
        .config("spark.blockManager.port", 10005) \
        .getOrCreate()

# RDD_API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

## Question A.1 ##

#Read file from hdfs
fileSV = spark_context.textFile('hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.sv')
fileEN = spark_context.textFile('hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.en')
linesSV = fileSV.count()
linesEN = fileEN.count()
print(f"Lines SV: {linesSV}, Lines EN: {linesEN}")
numPartitions = fileSV.getNumPartitions()
print(f"Number of partitions: {numPartitions}")

## Question A.2 ##

lowercaseSV = fileSV.map(lambda x: x.lower())
lowercaseEN = fileEN.map(lambda x: x.lower())
tokenSV = lowercaseSV.map(lambda x: x.split())
tokenEN = lowercaseEN.map(lambda x: x.split())
verifyLinesSV = tokenSV.count()
verifyLinesEN = tokenEN.count()
print(verifyLinesSV)
print(verifyLinesEN)

## Question A.3 ##

reTokenSV = lowercaseSV.flatMap(lambda x: re.findall(r"\b\w+\b", x))
reTokenEN = lowercaseEN.flatMap(lambda x: re.findall(r"\b\w+\b", x))
nWordsSV = reTokenSV.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
nWordsEN = reTokenEN.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
tenMostCommonSV = nWordsSV.sortBy(lambda x: x[1], ascending=False).take(10)
tenMostCommonEN = nWordsEN.sortBy(lambda x: x[1], ascending=False).take(10)
print(tenMostCommonSV)
print(tenMostCommonEN)

# The top ten most common words reflect what is reasonable in reality, words like is, or etc.
print("# The top ten most common words reflect what is reasonable in reality, words like is, or etc.")

## Question A.4 ##

sv_1 = tokenSV.zipWithIndex()
en_1 = tokenEN.zipWithIndex()
sv_2 = sv_1.map(lambda x: (x[1], x[0]))
en_2 = en_1.map(lambda x: (x[1], x[0]))
joined_3 = sv_2.join(en_2)
filter_4 = joined_3.filter(lambda x: x[1][0] and x[1][1])
filter_5 = filter_4.filter(lambda x: len(x[1][0]) < 7 and len(x[1][1]) < 7)
filter_6 = filter_5.filter(lambda x: len(x[1][0]) == len(x[1][1]))
zip_7 = filter_6.flatMap(lambda x: list(zip(x[1][0], x[1][1])))
reduced_8 = zip_7.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)
print_9 = reduced_8.sortBy(lambda x: x[1], ascending=False)
print_ = print_9.take(5)
print(print_)